## 使用pdfminer3k获取所有的行

In [8]:
from pdfminer.pdfparser import PDFParser,PDFDocument
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter,PDFTextExtractionNotAllowed
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams,LTTextLineHorizontal,LTFigure,LTRect,LTLine,LTCurve
 
#  文件对象
pd_file = open("D:/Reaserch/packing/packing 问题综述/test.pdf", "rb")
 
#  pdf文件解析对象
parser = PDFParser(pd_file)
 
# print(parser)
#  pdf文档对象
document = PDFDocument()
parser.set_document(document)
document.set_parser(parser)
 
 
#  初始化文档密码
document.initialize()
if document.is_extractable:
    print(True)
else:
    raise PDFTextExtractionNotAllowed
#  存储文档资源
src = PDFResourceManager()
 
#  设备对象
device = PDFPageAggregator(src,laparams=LAParams())
 
#  解释器对象
 
inter = PDFPageInterpreter(src,device)
 
pages = document.get_pages()
 
for page in pages:
    #print(page.contents)
    inter.process_page(page)
    layout = device.get_result()
    for x in layout:
        if isinstance(x, LTTextBoxHorizontal):
            parse(str(x.get_text()))

True


In [7]:
def parse(x):
    if(x=="References"):
        print(x)

In [9]:
import sys
import os.path
import re


def make_query(reference, l=3):
    f =  string.ascii_lowercase + ' '
    q = reference
    q = q.lower()
    q = q.replace('-', ' ')
    q = ''.join(c for c in q if c in f)
    q = q.split(' ')
    q = ' '.join(c for c in q if len(c) >= l)
    return q

def make_query_year(reference, l=3):
    f =  string.ascii_lowercase + ' '
    q = reference
    id_re = re.compile(r'(\d\d\d\d)')
    id = id_re.findall(q)
    year = id[0] if len(id) > 0 else '('
    q = q.split(year)
    q = q[1:]
    q = ''.join(q)
    q = q.lower()
    q = q.replace('-', ' ')
    q = ''.join(c for c in q if c in f)
    q = q.split(' ')
    q = ' '.join(c for c in q if len(c) >= l)
    if year != '(':
        q = year + ' ' + q
    return q

def get_articles(query, count=1):
    url = URL_SEARCH.format(q=query)
    soup = http_get(url)
    articles = []
    for tag in soup.findAll("div", { "class" : "gs_r" }):
        a = tag.find('a', text='Cite', attrs={ "class" : "gs_nph" })
        ident = re.search(r'gs_ocit\(event,\'(.*?)\',', a.get('onclick', '')).group(1)
        articles.append(ident)
    if count > 0:
        articles = articles[:count]
    return articles

def get_citations(ident, resolve=True):
    url = URL_CITE.format(ident=ident)
    soup = http_get(url)
    citations = {}
    for tag in soup.findAll('tr'):
        citations[tag.th.text] = tag.td.text
    for tag in soup.findAll('a', { "class": "gs_citi" }):
        citations[tag.text] = tag.get('href')
        if resolve:
            citations[tag.text] = http_get(citations[tag.text]).text
    return citations

# Put references here:
references = """Ribeiro, Bruno, and Towsley. Don. “Estimating and sampling graphs with multidimensional random walks”, Proceedings of the 10th ACM SIGCOMM conference on Internet measurement. ACM, 2010.
Wang, Tianyi, et al. “Understanding graph sampling algorithms for social network analysis”, 31st International Conference on Distributed Computing Systems Workshops. IEEE, 2011."""

myFile = open('D:\Reaserch\packing\packing 问题综述\bibliography.bib', 'w', encoding='utf-8')

for reference in references.split('\n'):
    reference = reference.strip()
    uprint ("\n---------------------------------------------------------")
    uprint ("Doing reference:", reference)
    q = make_query(reference)
    uprint ("Query used:", q)
    r = get_articles(q)
    if len(r) == 0:
        q = make_query_year(reference)
        r = get_articles(q)
    if len(r) == 0:
        uprint ("Still no results -- skipping")
        continue
    uprint ("Result written")
    myFile.write('\n\n%%% '+ r[0] + '  '+ q +'\n')
    myFile.write(get_citations(r[0]).get('BibTeX'))

myFile.close()

OSError: [Errno 22] Invalid argument: 'D:\\Reaserch\\packing\\packing 问题综述\x08ibliography.bib'